In [5]:
import os
import numpy as np
import pandas as pd
import csv
from datetime import datetime

#stay label
#labels for location
#column name:
#timestamp, newid, ID_Type, stay_lat, stay_lon, accuracy, stay_dur, stay_label, human_start_time
#source: https://github.com/feilongwang92/fhwa2020/tree/master/Mobility%20analysis

#test_stay = pd.read_csv(input_path+'stayonly_trip_identified_part_0.csv',header=None, sep='\t')

def get_cuebiq_trip_data(data_path):
    get_year = lambda x: datetime.fromtimestamp(x).year
    get_mes = lambda x: datetime.fromtimestamp(x).month
    get_day = lambda x: datetime.fromtimestamp(x).day
    get_weekday = lambda x: 1 if datetime.fromtimestamp(x).weekday() in [1,2,3] else 0
    
    test_stay = pd.read_csv(data_path, header=None, sep='\t')
    #drop columns not in used
    test_stay.pop(2)
    test_stay.pop(8)
    test_stay['year'] = test_stay[0].apply(get_year)
    test_stay['month'] = test_stay[0].apply(get_mes)
    test_stay['day'] = test_stay[0].apply(get_day)
    test_stay['weekday'] = test_stay[0].apply(get_weekday)

    test_stay = test_stay[test_stay['month']==4]
    test_stay = test_stay[test_stay['weekday']==1]
    print(test_stay.shape)
    if test_stay.shape[0]>0:
        return(test_stay)
    
def get_agg_trip_data(stay_csv_lst, input_path):
    index_ = 0 
    for stay_csv in stay_csv_lst:
        if index_ == 0:
            stay_trips = get_cuebiq_trip_data(input_path+stay_csv)
            index_ += 1
        else:
            stay_trips_cur = get_cuebiq_trip_data(input_path+stay_csv)
            stay_trips = pd.concat([stay_trips, stay_trips_cur])
    print(stay_trips.shape)
    return(stay_trips)

#cuebiq stay trip generation
input_path = 'G:/Shared drives/Thinklab with Tan/data/'
stay_csv_lst = []
for i in os.listdir(input_path):
    if 'identified_part' in i and 'sample' not in i:
        print(i)
        stay_csv_lst.append(i)
        
stay_trips = get_agg_trip_data(stay_csv_lst, input_path)
stay_trips.to_csv('G:/My Drive/2021/Bias/SUMO_simulation/appsim/cuebiq_stay_trip.csv')